# 🎤 演講逐字稿生成器 | Presentation Transcript Generator

> **專業級投影片逐字稿生成工具** - 運用 AI Agent 技術，將投影片轉換為自然流暢的演講逐字稿

---

## ✨ 功能特色

- 📊 **智能投影片分析** - 自動解析 PDF 投影片內容
- 🎙️ **語速智能偵測** - 上傳 20 秒音頻自動計算語速
- 🎭 **多樣演講風格** - 支援活潑、嚴肅等多種風格
- 🌐 **多語言支援** - 繁體中文、英文等多種語言
- 👨‍🏫 **專家角色扮演** - AI 扮演領域專家生成專業內容
- 📥 **一鍵下載** - 支援多種格式導出

---

## 🚀 使用流程

1. **環境設置** - 安裝必要套件
2. **配置參數** - 設定演講時間、風格、語言等
3. **上傳文件** - 上傳投影片 PDF 和音頻（選填）
4. **生成逐字稿** - AI 自動生成專業逐字稿
5. **下載結果** - 取得完整逐字稿文件

---

**製作者**: 使用頂尖程式設計理念與用戶體驗設計  
**最後更新**: 2025年12月

In [ ]:
%%capture
# 安裝必要套件
!pip install -q pymupdf pillow
!pip install -q pydub
!pip install -q openai
!pip install -q ipywidgets
!apt-get install -qq ffmpeg

print("✅ 所有套件安裝完成！")

In [ ]:
# 導入標準庫
import os
import io
import json
import base64
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Optional, Tuple

# 導入第三方庫
import fitz  # PyMuPDF
from PIL import Image
from pydub import AudioSegment
from openai import OpenAI
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# 設定警告
import warnings
warnings.filterwarnings('ignore')

print("✅ 模組導入成功！")
print("📌 請在下一步設定您的 API Key")

In [ ]:
# 設定 CSS 樣式 - 使用 Noto Sans TC 替代新細明體
custom_css = """
<style>
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+TC:wght@300;400;500;700&display=swap');

* {
    font-family: 'Noto Sans TC', 'Segoe UI', Arial, sans-serif !important;
}

.widget-label {
    font-weight: 500 !important;
    color: #2c3e50 !important;
    font-size: 14px !important;
}

.widget-text input, .widget-textarea textarea, .widget-dropdown select {
    border: 2px solid #e0e0e0 !important;
    border-radius: 8px !important;
    padding: 10px !important;
    font-size: 14px !important;
    transition: all 0.3s ease !important;
}

.widget-text input:focus, .widget-textarea textarea:focus, .widget-dropdown select:focus {
    border-color: #4CAF50 !important;
    box-shadow: 0 0 0 3px rgba(76, 175, 80, 0.1) !important;
    outline: none !important;
}

.widget-button {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    color: white !important;
    border: none !important;
    border-radius: 8px !important;
    padding: 12px 24px !important;
    font-weight: 500 !important;
    font-size: 14px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
}

.widget-button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4) !important;
}

.success-box {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 12px;
    margin: 20px 0;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
}

.info-box {
    background: #f8f9fa;
    border-left: 4px solid #667eea;
    padding: 15px;
    border-radius: 8px;
    margin: 15px 0;
}

.transcript-output {
    background: white;
    border: 2px solid #e0e0e0;
    border-radius: 12px;
    padding: 25px;
    margin: 20px 0;
    box-shadow: 0 2px 10px rgba(0,0,0,0.05);
    max-height: 500px;
    overflow-y: auto;
}

.slide-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 12px 20px;
    border-radius: 8px;
    font-weight: 600;
    font-size: 16px;
    margin-top: 20px;
    margin-bottom: 10px;
}

.slide-content {
    color: #2c3e50;
    line-height: 1.8;
    font-size: 15px;
    padding: 10px 20px;
}

.progress-indicator {
    background: #f8f9fa;
    border-radius: 12px;
    padding: 20px;
    margin: 15px 0;
    border: 2px solid #e0e0e0;
}

h1, h2, h3 {
    color: #2c3e50 !important;
    font-weight: 600 !important;
}
</style>
"""

display(HTML(custom_css))
print("✅ UI 樣式設定完成！採用 Noto Sans TC 字體")

In [ ]:
class PDFProcessor:
    """處理 PDF 投影片的類別"""
    
    def __init__(self):
        self.slides_content = []
    
    def extract_slides(self, pdf_path: str) -> List[Dict[str, str]]:
        """
        從 PDF 提取每一頁的內容
        
        Args:
            pdf_path: PDF 檔案路徑
            
        Returns:
            每頁內容的列表 [{"page": 1, "text": "...", "image": "..."}, ...]
        """
        try:
            doc = fitz.open(pdf_path)
            
            # 檢查是否為空 PDF
            if len(doc) == 0:
                doc.close()
                raise Exception("此 PDF 檔案不包含任何頁面")
            
            slides = []
            
            for page_num in range(len(doc)):
                page = doc[page_num]
                
                # 提取文字
                text = page.get_text().strip()
                
                # 轉換為圖片（用於視覺化或OCR）
                pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))
                img_data = pix.tobytes("png")
                
                slides.append({
                    "page": page_num + 1,
                    "text": text if text else "[此頁無文字內容]",
                    "image": base64.b64encode(img_data).decode()
                })
            
            doc.close()
            self.slides_content = slides
            return slides
            
        except Exception as e:
            if "PDF" in str(e):
                raise Exception(f"PDF 處理錯誤: {str(e)}")
            else:
                raise Exception(f"PDF 處理錯誤: 無法讀取檔案，請確認檔案格式正確")
    
    def get_slide_summary(self) -> str:
        """取得投影片摘要"""
        if not self.slides_content:
            return "尚未載入投影片"
        
        summary = f"共 {len(self.slides_content)} 頁投影片\n\n"
        for slide in self.slides_content[:3]:  # 顯示前3頁預覽
            summary += f"📄 第 {slide['page']} 頁:\n{slide['text'][:100]}...\n\n"
        
        if len(self.slides_content) > 3:
            summary += f"...以及其他 {len(self.slides_content) - 3} 頁"
        
        return summary


class AudioAnalyzer:
    """使用 GPT-4o Audio API 分析音頻並計算語速"""
    
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.words_per_minute = None
    
    def _convert_m4a_to_mp3(self, audio_path: str) -> str:
        """將 m4a 格式轉換為 mp3"""
        try:
            audio = AudioSegment.from_file(audio_path)
            mp3_path = "/tmp/converted_audio.mp3"
            audio.export(mp3_path, format="mp3", bitrate="128k")
            return mp3_path
        except Exception as e:
            raise Exception(f"音頻格式轉換錯誤: {str(e)}")
    
    def analyze_audio(self, audio_path: str) -> float:
        """使用 GPT-4o Audio API 分析音頻並計算語速"""
        try:
            audio = AudioSegment.from_file(audio_path)
            duration_seconds = len(audio) / 1000.0
            
            # 檢查音頻時長
            if duration_seconds < 5:
                raise Exception("音頻時長過短（少於 5 秒），建議上傳 20 秒左右的音頻以獲得更準確的結果")
            if duration_seconds > 120:
                raise Exception("音頻時長過長（超過 2 分鐘），請上傳 20-60 秒的音頻樣本")
            
            # 如果是 m4a，轉換為 mp3
            if audio_path.lower().endswith('.m4a'):
                print("🔄 偵測到 m4a 格式，正在轉換為 mp3...")
                audio_path = self._convert_m4a_to_mp3(audio_path)
            
            # 使用 GPT-4o Audio API 進行轉錄
            print("🎙️ 使用 GPT-4o Audio API 分析中...")
            
            with open(audio_path, 'rb') as audio_file:
                transcription = self.client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    language="zh"
                )
            
            text = transcription.text
            
            # 檢查是否有轉錄內容
            if not text or len(text.strip()) == 0:
                raise Exception("無法識別音頻內容，請確保音頻清晰且包含語音")
            
            # 計算字數（中文按字數計算）
            char_count = len([c for c in text if c.strip() and not c.isspace()])
            
            # 計算每分鐘字數
            wpm = (char_count / duration_seconds) * 60
            self.words_per_minute = wpm
            
            # 清理暫存檔
            if audio_path.startswith("/tmp/"):
                if os.path.exists(audio_path):
                    os.remove(audio_path)
            
            return wpm
            
        except Exception as e:
            # 清理暫存檔（即使發生錯誤）
            if 'audio_path' in locals() and audio_path.startswith("/tmp/"):
                if os.path.exists(audio_path):
                    os.remove(audio_path)
            raise Exception(f"音頻分析錯誤: {str(e)}")


class TranscriptGenerator:
    """使用 OpenAI Vision 模型生成演講逐字稿（支援 GPT-5.1/o3/GPT-4o/GPT-4o-mini）"""
    
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        self.transcript = ""
        self.use_vision = True
    
    def generate_transcript(
        self,
        slides: List[Dict[str, str]],
        target_duration: int,
        words_per_minute: float,
        style: str,
        topic: str,
        audience: str,
        language: str,
        model_name: str = "gpt-5.1",
        expert_role: Optional[str] = None,
        include_tips: bool = False
    ) -> str:
        """生成演講逐字稿（支援多語言、多模型和演講技巧建議）"""
        try:
            # 檢查投影片是否為空
            if not slides or len(slides) == 0:
                raise Exception("沒有投影片內容，請先上傳 PDF 檔案")
            
            # 計算目標字數
            target_words = int(target_duration * words_per_minute)
            words_per_slide = target_words // len(slides)
            
            # 建立系統提示
            system_prompt = self._create_system_prompt(
                style, topic, audience, language, expert_role, words_per_slide, include_tips
            )
            
            # 顯示生成資訊
            model_names = {
                'gpt-5.1': 'GPT-5.1（最強多模態理解）',
                'o3': 'o3（強推理模型）',
                'gpt-4o': 'GPT-4o（全能平衡）',
                'gpt-4o-mini': 'GPT-4o-mini（快速經濟）'
            }
            print(f"🤖 使用 {model_names.get(model_name, model_name)} 生成逐字稿...")
            print(f"📊 目標字數: {target_words} 字")
            print(f"📄 投影片數量: {len(slides)} 頁")
            print(f"🌐 輸出語言: {language}")
            if include_tips:
                print("💡 包含演講技巧建議（手勢、語氣、暫停等）")
            
            # 建立提示內容
            tips_instruction = ""
            if include_tips:
                tips_instruction = """

【演講技巧建議】
請在逐字稿中適當位置加入以下演講技巧建議（使用 [方括號] 標註）：
- [手勢：展開雙手] - 在強調重點時
- [手勢：指向投影片] - 在說明圖表時
- [語氣：提高音量] - 在關鍵訊息時
- [語氣：放慢速度] - 在重要概念時
- [暫停 2-3 秒] - 在段落轉換時
- [眼神接觸] - 在與聽眾互動時
- [走動：移向舞台中央] - 在開場或總結時
"""
            
            user_content = [
                {
                    "type": "text",
                    "text": f"""
請根據以下投影片圖片，生成一份完整的演講逐字稿。

演講參數：
- 總時長：{target_duration} 分鐘
- 語速：每分鐘約 {int(words_per_minute)} 字
- 目標總字數：約 {target_words} 字
- 每頁建議字數：約 {words_per_slide} 字
- 輸出語言：{language}{tips_instruction}

輸出格式要求：
Slide 1
[第一頁的演講內容]

Slide 2
[第二頁的演講內容]

...以此類推

請確保：
1. 仔細觀察每頁投影片的視覺元素、圖表、文字
2. 每頁的逐字稿自然流暢，講解投影片上的重點
3. 內容銜接順暢，有開場和結尾
4. 符合指定的演講風格和語氣
5. 總字數控制在 {target_words} 字左右（可上下浮動10%）
"""
                }
            ]
            
            # 添加所有投影片圖片
            for slide in slides:
                user_content.append({
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{slide['image']}",
                        "detail": "high"
                    }
                })
            # 呼叫 OpenAI Vision API（支援 GPT-5.1/o3/GPT-4o/GPT-4o-mini）
            response = self.client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_content}
                ],
                temperature=0.7,
                max_completion_tokens=4000
            )
            
            transcript = response.choices[0].message.content
            self.transcript = transcript
            
            return transcript
            
        except Exception as e:
            error_msg = str(e)
            if "API key" in error_msg or "authentication" in error_msg.lower():
                raise Exception("❌ API Key 錯誤，請確認您的 OpenAI API Key 是否正確")
            elif "rate limit" in error_msg.lower():
                raise Exception("❌ API 請求過於頻繁，請稍後再試")
            elif "quota" in error_msg.lower():
                raise Exception("❌ API 額度不足，請檢查您的 OpenAI 帳戶餘額")
            else:
                raise Exception(f"逐字稿生成錯誤: {error_msg}")
    
    def _create_system_prompt(
        self,
        style: str,
        topic: str,
        audience: str,
        language: str,
        expert_role: Optional[str],
        words_per_slide: int,
        include_tips: bool = False
    ) -> str:
        """建立系統提示"""
        
        style_descriptions = {
            "活潑": "使用輕鬆、活潑的語氣，適度加入互動和幽默元素",
            "嚴肅": "使用正式、專業的語氣，保持學術嚴謹性",
            "激勵": "使用鼓舞人心的語言，充滿正能量和動力",
            "教學": "使用清晰、易懂的解釋方式，像是在教導學生",
            "對話": "使用對話式的語氣，如同與聽眾面對面交談"
        }
        
        language_instructions = {
            "繁體中文": "使用繁體中文輸出",
            "英文": "使用英文輸出",
            "簡體中文": "使用簡體中文輸出",
            "日文": "使用日文輸出",
            "韓文": "使用韓文輸出",
            "西班牙文": "使用西班牙文輸出",
            "法文": "使用法文輸出",
            "德文": "使用德文輸出"
        }
        
        role_intro = ""
        if expert_role:
            role_intro = f"你是一位{expert_role}，"
        
        style_desc = style_descriptions.get(style, "使用自然流暢的語氣")
        lang_inst = language_instructions.get(language, "使用繁體中文輸出")
        
        tips_requirement = ""
        if include_tips:
            tips_requirement = """
7. 在適當位置加入演講技巧建議，使用 [方括號] 標註，包括：
   - 手勢建議（如：展開雙手、指向投影片、握拳強調）
   - 語氣建議（如：提高音量、放慢速度、加重語氣）
   - 暫停時機（如：[暫停 2-3 秒]）
   - 肢體語言（如：眼神接觸、走動、身體前傾）
   這些建議應自然融入逐字稿中，幫助演講者更好地傳達訊息
"""
        
        return f"""
{role_intro}你是一位經驗豐富的演講者和內容創作專家。

演講主題：{topic}
目標聽眾：{audience}
演講風格：{style_desc}
語言要求：{lang_inst}

你的任務是根據提供的投影片內容，創作一份自然、流暢、引人入勝的演講逐字稿。

要求：
1. 內容必須忠於投影片，但要用口語化的方式表達
2. 每頁約 {words_per_slide} 字，可根據內容重要性調整
3. 開場要吸引人，結尾要有力
4. 適時加入過渡語，讓內容銜接流暢
5. 符合指定的演講風格和目標聽眾
6. 確保內容專業準確，同時易於理解{tips_requirement}
"""

print("✅ 核心功能類別已建立！")

In [ ]:
from google.colab import userdata
import getpass

# 嘗試從 Colab Secrets 讀取
try:
    OPENAI_API_KEY = userdata.get('GPT_API_KEY')
    print("✅ 已從 Colab Secrets 載入 API Key")
except:
    # 手動輸入
    print("🔑 請輸入您的 OpenAI API Key:")
    print("💡 提示：可在 Colab 的「Secrets」功能中儲存 API Key")
    OPENAI_API_KEY = getpass.getpass("API Key: ")
    
if OPENAI_API_KEY:
    print("✅ API Key 設定完成！")
else:
    print("⚠️ 警告：未設定 API Key，無法使用 AI 生成功能")

In [ ]:
class TranscriptGeneratorUI:
    """互動式使用者界面"""
    
    def __init__(self):
        self.pdf_processor = PDFProcessor()
        self.audio_analyzer = None  # 延遲初始化，需要 API key
        self.transcript_generator = None
        
        # 儲存上傳的檔案
        self.pdf_path = None
        self.audio_path = None
        self.current_wpm = 200  # 當前語速
        
        # 建立 UI 元件
        self._create_widgets()
    
    def _create_widgets(self):
        """建立所有 UI 元件"""
        
        # 標題
        display(HTML("""
        <div class="success-box">
            <h2 style="color: white; margin: 0;">🎤 演講逐字稿生成器</h2>
            <p style="color: white; margin: 10px 0 0 0; opacity: 0.9;">
                輕鬆將投影片轉換為專業演講逐字稿
            </p>
        </div>
        """))
        
        # 1. 上傳 PDF
        display(HTML("<div class='info-box'><h3>📄 步驟 1: 上傳投影片 PDF</h3></div>"))
        self.pdf_upload = widgets.FileUpload(
            accept='.pdf',
            multiple=False,
            description='選擇 PDF'
        )
        self.pdf_status = widgets.HTML(value="<p style='color: #666;'>尚未上傳</p>")
        display(self.pdf_upload, self.pdf_status)
        
        # 2. 設定演講時間
        display(HTML("<div class='info-box'><h3>⏱️ 步驟 2: 設定演講時間</h3></div>"))
        self.duration_input = widgets.IntText(
            value=10,
            description='演講時長',
            min=1,
            max=180,
            style={'description_width': 'initial'}
        )
        display(widgets.HBox([self.duration_input, widgets.Label('分鐘')]))
        
        # 3. 語速設定
        display(HTML("<div class='info-box'><h3>🎙️ 步驟 3: 設定語速</h3></div>"))
        
        # 語速選擇下拉選單
        self.speed_preset = widgets.Dropdown(
            options=[
                ('慢速 (150 字/分)', 150),
                ('中速 (200 字/分)', 200),
                ('快速 (250 字/分)', 250),
                ('自動分析 (上傳 20 秒音頻)', 0)
            ],
            value=200,
            description='語速',
            style={'description_width': 'initial'}
        )
        display(self.speed_preset)
        
        # 音頻上傳區域（當選擇「自動分析」時使用）
        self.audio_container = widgets.VBox([
            widgets.HTML("<p style='color: #666; font-size: 13px; margin: 10px 0;'>💡 選擇「自動分析」後，請上傳 20 秒音頻</p>")
        ])
        
        self.audio_upload = widgets.FileUpload(
            accept='.m4a,.mp3,.wav',
            multiple=False,
            description='上傳音頻',
            layout=widgets.Layout(display='none')  # 預設隱藏
        )
        self.audio_status = widgets.HTML(value="")
        self.analyze_button = widgets.Button(
            description='🎵 開始分析',
            button_style='info',
            layout=widgets.Layout(display='none')  # 預設隱藏
        )
        
        self.audio_container.children = self.audio_container.children + (self.audio_upload, self.audio_status, self.analyze_button)
        display(self.audio_container)
        
        # 監聽語速選擇變化
        self.speed_preset.observe(self._on_speed_change, names='value')
        
        # 🆕 4. AI 模型選擇
        display(HTML("""
        <div class='info-box'>
            <h3>🤖 步驟 4: 選擇 AI 模型</h3>
            <p style='color: #666; font-size: 13px; margin: 5px 0;'>
                💡 <strong>GPT-5.1</strong> 具備最強大的多模態理解能力，能深度分析圖片與文字
            </p>
        </div>
        """))
        self.model_dropdown = widgets.Dropdown(
            options=[
                ('GPT-5.1 - 最強多模態（圖文深度理解，建議）⭐', 'gpt-5.1'),
                ('o3 - 強推理（複雜邏輯分析）', 'o3'),
                ('GPT-4o - 全能平衡（速度與品質兼顧）', 'gpt-4o'),
                ('GPT-4o-mini - 快速經濟（基礎需求）', 'gpt-4o-mini')
            ],
            value='gpt-5.1',
            description='AI 模型',
            style={'description_width': 'initial'}
        )
        display(self.model_dropdown)
        
        # 5. 演講風格
        display(HTML("<div class='info-box'><h3>🎭 步驟 5: 選擇演講風格</h3></div>"))
        self.style_dropdown = widgets.Dropdown(
            options=['活潑', '嚴肅', '激勵', '教學', '對話'],
            value='活潑',
            description='演講風格',
            style={'description_width': 'initial'}
        )
        display(self.style_dropdown)
        
        # 6. 演講資訊
        display(HTML("<div class='info-box'><h3>📝 步驟 6: 填寫演講資訊</h3></div>"))
        
        self.topic_input = widgets.Text(
            value='',
            placeholder='例如：人工智慧在教育中的應用',
            description='演講題目',
            style={'description_width': 'initial'}
        )
        
        self.audience_input = widgets.Text(
            value='',
            placeholder='例如：大學生、教師、科技愛好者',
            description='目標聽眾',
            style={'description_width': 'initial'}
        )
        
        # 語言選項
        self.language_dropdown = widgets.Dropdown(
            options=['繁體中文', '英文', '簡體中文', '日文', '韓文', '西班牙文', '法文', '德文'],
            value='繁體中文',
            description='輸出語言',
            style={'description_width': 'initial'}
        )
        
        # 🆕 專家角色 - 添加詳細說明
        display(HTML("""
        <div style='background: #E3F2FD; border-left: 4px solid #2196F3; padding: 12px; border-radius: 6px; margin: 10px 0;'>
            <strong style='color: #1976D2;'>💡 專家角色是什麼？</strong>
            <p style='color: #555; font-size: 13px; margin: 8px 0 0 0; line-height: 1.6;'>
                AI 會<strong>扮演您指定的專家身份</strong>來撰寫逐字稿，使內容更專業、更有說服力。<br>
                • 例如：「資深AI研究員」→ 用技術專家的角度講解<br>
                • 例如：「教育心理學博士」→ 用教育專家的視角闡述<br>
                • 例如：「創業導師」→ 用實戰經驗分享觀點<br>
                <em>留空則使用通用演講者身份</em>
            </p>
        </div>
        """))
        
        self.expert_role = widgets.Text(
            value='',
            placeholder='例如：資深AI研究員、教育心理學博士、創業導師（選填）',
            description='專家角色',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )
        
        # 演講技巧建議選項
        self.include_tips = widgets.Checkbox(
            value=True,
            description='包含演講技巧建議（手勢、語氣、暫停等）',
            style={'description_width': 'initial'}
        )
        
        display(self.topic_input, self.audience_input, self.language_dropdown, self.expert_role, self.include_tips)
        
        # 7. 生成按鈕
        display(HTML("<div style='margin-top: 30px;'></div>"))
        self.generate_button = widgets.Button(
            description='🚀 生成逐字稿',
            button_style='success',
            layout=widgets.Layout(width='200px', height='50px')
        )
        display(self.generate_button)
        
        # 8. 輸出區域
        self.output_area = widgets.Output()
        display(self.output_area)
        
        # 綁定事件
        self.pdf_upload.observe(self._on_pdf_upload, names='value')
        self.audio_upload.observe(self._on_audio_upload, names='value')
        self.analyze_button.on_click(self._analyze_audio)
        self.generate_button.on_click(self._generate_transcript)
    
    def _on_speed_change(self, change):
        """處理語速選擇變化"""
        if change['new'] == 0:  # 選擇了「自動分析」
            self.audio_upload.layout.display = 'block'
            self.analyze_button.layout.display = 'block'
            self.audio_status.value = "<p style='color: #2196F3;'>📤 請上傳 20 秒音頻檔案</p>"
        else:
            self.audio_upload.layout.display = 'none'
            self.analyze_button.layout.display = 'none'
            self.audio_status.value = ""
            self.current_wpm = change['new']
    
    def _on_pdf_upload(self, change):
        """處理 PDF 上傳"""
        if change['new']:
            try:
                uploaded_file = list(change['new'].values())[0]
                filename = uploaded_file['metadata']['name']
                file_size = len(uploaded_file['content'])
                
                # 檢查檔案大小（建議 < 50MB）
                if file_size > 50 * 1024 * 1024:
                    self.pdf_status.value = "<div style='color: #f44336;'>❌ 檔案過大（超過 50MB），請壓縮後再上傳</div>"
                    return
                
                # 儲存 PDF
                self.pdf_path = "/tmp/presentation.pdf"
                with open(self.pdf_path, 'wb') as f:
                    f.write(uploaded_file['content'])
                
                # 解析 PDF
                slides = self.pdf_processor.extract_slides(self.pdf_path)
                
                self.pdf_status.value = f"""
                <div style='color: #4CAF50; font-weight: 500;'>
                    ✅ 已上傳：{filename}<br>
                    📊 共 {len(slides)} 頁投影片
                </div>
                """
                
            except Exception as e:
                self.pdf_status.value = f"<div style='color: #f44336;'>❌ 錯誤：{str(e)}</div>"
    
    def _on_audio_upload(self, change):
        """處理音頻上傳"""
        if change['new']:
            try:
                uploaded_file = list(change['new'].values())[0]
                
                # 儲存音頻
                filename = uploaded_file['metadata']['name']
                ext = os.path.splitext(filename)[1]
                self.audio_path = f"/tmp/audio{ext}"
                
                with open(self.audio_path, 'wb') as f:
                    f.write(uploaded_file['content'])
                
                self.audio_status.value = f"""
                <div style='color: #4CAF50; font-weight: 500;'>
                    ✅ 已上傳：{filename}<br>
                    👉 請點擊「開始分析」按鈕
                </div>
                """
                
            except Exception as e:
                self.audio_status.value = f"<div style='color: #f44336;'>❌ 錯誤：{str(e)}</div>"
    
    def _analyze_audio(self, button):
        """使用 GPT-4o Audio API 分析音頻語速"""
        if not self.audio_path:
            self.audio_status.value = "<div style='color: #f44336;'>❌ 請先上傳音頻檔案</div>"
            return
        
        try:
            self.audio_status.value = "<div style='color: #2196F3;'>⏳ 使用 GPT-4o Audio API 分析中...</div>"
            
            # 初始化 AudioAnalyzer（需要 API key）
            if self.audio_analyzer is None:
                self.audio_analyzer = AudioAnalyzer(OPENAI_API_KEY)
            
            wpm = self.audio_analyzer.analyze_audio(self.audio_path)
            
            # 更新當前語速
            self.current_wpm = int(wpm)
            
            self.audio_status.value = f"""
            <div style='color: #4CAF50; font-weight: 500;'>
                ✅ 分析完成！<br>
                🎤 您的語速：{self.current_wpm} 字/分鐘
            </div>
            """
            
        except Exception as e:
            self.audio_status.value = f"<div style='color: #f44336;'>❌ {str(e)}</div>"
    
    def _generate_transcript(self, button):
        """生成逐字稿"""
        with self.output_area:
            clear_output()
            
            # 驗證輸入
            if not self.pdf_path:
                display(HTML("<div style='color: #f44336;'>❌ 請先上傳 PDF 投影片</div>"))
                return
            
            if not self.topic_input.value:
                display(HTML("<div style='color: #f44336;'>❌ 請填寫演講題目</div>"))
                return
            
            if not self.audience_input.value:
                display(HTML("<div style='color: #f44336;'>❌ 請填寫目標聽眾</div>"))
                return
            
            try:
                display(HTML("""
                <div class='progress-indicator'>
                    <h3>🔄 正在生成逐字稿...</h3>
                    <p>請稍候，這可能需要一些時間。</p>
                </div>
                """))
                
                # 初始化生成器
                self.transcript_generator = TranscriptGenerator(OPENAI_API_KEY)
                
                # 決定使用的語速
                if self.speed_preset.value == 0:  # 自動分析
                    if self.current_wpm == 200:  # 尚未分析
                        clear_output()
                        display(HTML("<div style='color: #f44336;'>❌ 請先上傳音頻並完成分析，或選擇預設語速</div>"))
                        return
                    wpm = self.current_wpm
                else:
                    wpm = self.speed_preset.value
                
                # 🆕 生成逐字稿（傳遞模型選擇）
                transcript = self.transcript_generator.generate_transcript(
                    slides=self.pdf_processor.slides_content,
                    target_duration=self.duration_input.value,
                    words_per_minute=wpm,
                    style=self.style_dropdown.value,
                    topic=self.topic_input.value,
                    audience=self.audience_input.value,
                    language=self.language_dropdown.value,
                    model_name=self.model_dropdown.value,
                    expert_role=self.expert_role.value if self.expert_role.value else None,
                    include_tips=self.include_tips.value
                )
                
                clear_output()
                
                # 顯示結果
                display(HTML("""
                <div class='success-box'>
                    <h2 style="color: white; margin: 0;">✅ 逐字稿生成完成！</h2>
                </div>
                """))
                
                # 格式化輸出
                formatted_transcript = self._format_transcript(transcript)
                display(HTML(f"<div class='transcript-output'>{formatted_transcript}</div>"))
                
                # 下載按鈕
                self._create_download_buttons(transcript)
                
            except Exception as e:
                clear_output()
                display(HTML(f"""
                <div style='color: #f44336; padding: 20px; border: 2px solid #f44336; border-radius: 8px;'>
                    <h3>❌ 生成失敗</h3>
                    <p>{str(e)}</p>
                </div>
                """))
    
    def _format_transcript(self, transcript: str) -> str:
        """格式化逐字稿輸出"""
        lines = transcript.split('\n')
        formatted = ""
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # 投影片標題
            if line.startswith('Slide'):
                formatted += f"<h3 style='color: #2196F3; margin-top: 20px; border-left: 4px solid #2196F3; padding-left: 10px;'>📄 {line}</h3>"
            # 演講技巧建議（方括號標註）
            elif '[' in line and ']' in line:
                # 高亮顯示建議
                import re
                highlighted = re.sub(r'\[([^\]]+)\]', r'<span style="background: #FFF3E0; color: #F57C00; padding: 2px 6px; border-radius: 3px; font-weight: 500;">[\1]</span>', line)
                formatted += f"<p style='line-height: 1.8; margin: 10px 0;'>{highlighted}</p>"
            # 一般內容
            else:
                formatted += f"<p style='line-height: 1.8; margin: 10px 0;'>{line}</p>"
        
        return formatted
    
    def _create_download_buttons(self, transcript: str):
        """建立下載按鈕"""
        
        # TXT 下載
        txt_content = transcript.encode('utf-8')
        txt_b64 = base64.b64encode(txt_content).decode()
        
        filename = f"transcript_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        display(HTML(f"""
        <div style='margin-top: 20px; text-align: center;'>
            <a href="data:text/plain;base64,{txt_b64}" 
               download="{filename}.txt"
               style="display: inline-block; padding: 12px 30px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                      color: white; text-decoration: none; border-radius: 8px; font-weight: 500; margin: 10px;">
                📥 下載 TXT 格式
            </a>
        </div>
        """))

print("✅ 互動式 UI 已準備就緒！")
print("👇 請向下滾動開始使用")

In [ ]:
# 顯示啟動訊息
print("\n" + "="*80)
print("🎉 應用程式啟動中...")
print("="*80)
print("\n📋 使用步驟：")
print("1️⃣  上傳您的投影片 PDF")
print("2️⃣  設定演講時長")
print("3️⃣  選擇語速（慢/中/快/自動分析）")
print("4️⃣  選擇 AI 模型（GPT-5.1 推薦 ⭐）")
print("5️⃣  選擇演講風格")
print("6️⃣  填寫演講資訊")
print("7️⃣  點擊「生成逐字稿」按鈕")
print("8️⃣  下載生成的逐字稿")
print("\n✨ 核心特色：")
print("   • GPT-5.1 模型 - 最強多模態理解，圖文深度分析投影片內容")
print("   • GPT-4o Audio - 精準計算您的語速")
print("   • 8 種語言支援 - 繁中/英/簡中/日/韓/西/法/德")
print("   • 演講技巧建議 - AI 提供手勢、語氣、暫停等專業建議")
print("   • 專家角色扮演 - AI 以指定專家身份撰寫，內容更專業有說服力")
print("\n💡 專家角色說明：")
print("   填寫「專家角色」欄位後，AI會以該身份撰寫逐字稿")
print("   例如：「資深AI研究員」會用技術專家角度講解")
print("        「教育心理學博士」會用教育專家視角闡述")
print("="*80 + "\n")

# 啟動應用程式
app = TranscriptGeneratorUI()